In [3]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 3.6 MB/s 
     |████████████████████████████████| 40 kB 6.8 MB/s 
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
import tensorflow as tf
import tensorflowjs as tfjs
import pandas as pd
import numpy as np
import json

In [22]:
trainsplit = 7000
emotionDict = {'joy': 0, 'fear': 1, 'anger': 2, 'sadness': 3, 'disgust': 4, 'shame': 5, 'guilt': 6}

dataFrame = pd.read_csv('ISEAR.csv')
for i in range(len(dataFrame['emotion'])):
    dataFrame['emotion'][i] = emotionDict[dataFrame['emotion'][i]]
xTrain = dataFrame['sentence'][:trainsplit]
yTrain = dataFrame['emotion'][:trainsplit]
xTest = dataFrame['sentence'][trainsplit:]
yTest = dataFrame['emotion'][:trainsplit]
dataFrame.tail()

,emotion,sentence
7516,5,Two years back someone invited me to be the tu...
7517,5,I had taken the responsibility to do something...
7518,1,I was at home and I heard a loud sound of spit...
7519,6,I did not do the homework that the teacher had...
7520,1,I had shouted at my younger brother and he was...


In [23]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(xTrain)
vocab = tokenizer.word_index
vocabSize = len(vocab)
print(vocabSize)
f = open('../src/stokenizer.json', 'w')
f.write(json.dumps(vocab))
f.close()

8796


In [29]:
vocab

{'<shobdobhandarErBahire>': 1,
 'i': 2,
 'a': 3,
 'the': 4,
 'my': 5,
 'to': 6,
 'was': 7,
 'and': 8,
 'when': 9,
 'of': 10,
 'in': 11,
 'had': 12,
 'me': 13,
 'that': 14,
 'for': 15,
 'with': 16,
 'it': 17,
 'at': 18,
 'not': 19,
 'on': 20,
 'he': 21,
 'very': 22,
 'friend': 23,
 'felt': 24,
 'an': 25,
 'she': 26,
 'her': 27,
 'one': 28,
 'we': 29,
 'about': 30,
 'from': 31,
 'after': 32,
 'as': 33,
 'this': 34,
 'by': 35,
 'time': 36,
 'were': 37,
 'out': 38,
 'who': 39,
 'but': 40,
 'because': 41,
 'did': 42,
 'him': 43,
 'got': 44,
 'his': 45,
 'been': 46,
 'which': 47,
 'home': 48,
 'have': 49,
 'some': 50,
 'up': 51,
 'friends': 52,
 'told': 53,
 'mother': 54,
 'school': 55,
 'day': 56,
 'be': 57,
 'so': 58,
 'people': 59,
 'would': 60,
 'they': 61,
 'went': 62,
 'do': 63,
 'person': 64,
 'there': 65,
 'saw': 66,
 'is': 67,
 'first': 68,
 'father': 69,
 'came': 70,
 'mine': 71,
 "didn't": 72,
 'our': 73,
 'all': 74,
 'car': 75,
 'night': 76,
 'girl': 77,
 'could': 78,
 'having': 

In [33]:
xTrain = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataFrame['sentence']), padding='pre', maxlen=200)[:trainsplit]
yTrain = tf.keras.utils.to_categorical((dataFrame['emotion'][:trainsplit]))
xTest = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataFrame['sentence']), padding='pre', maxlen=200)[trainsplit:]
yTest = tf.keras.utils.to_categorical((dataFrame['emotion'][trainsplit:]))
xTrain.shape, yTrain.shape, vocabSize

((7000, 200), (7000, 7), 8796)

In [72]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 128, input_length=200))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy' , optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [74]:
model.fit(xTrain, yTrain, epochs=15, shuffle=True)

Epoch 1/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0630 - accuracy: 0.9789
Epoch 2/15
219/219 [==============================] - 5s 24ms/step - loss: 0.0945 - accuracy: 0.9720
Epoch 3/15
219/219 [==============================] - 5s 24ms/step - loss: 0.0677 - accuracy: 0.9779
Epoch 4/15
219/219 [==============================] - 5s 24ms/step - loss: 0.0491 - accuracy: 0.9853
Epoch 5/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0326 - accuracy: 0.9906
Epoch 6/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0317 - accuracy: 0.9897
Epoch 7/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0485 - accuracy: 0.9857
Epoch 8/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0432 - accuracy: 0.9869
Epoch 9/15
219/219 [==============================] - 5s 24ms/step - loss: 0.0330 - accuracy: 0.9890
Epoch 10/15
219/219 [==============================] - 5s 25ms/step - loss: 0.0338 - accura

In [75]:
model.evaluate(xTest, yTest)

17/17 [==============================] - 1s 19ms/step - loss: 3.4706 - accuracy: 0.5240


[3.4705631732940674, 0.5239923000335693]

In [77]:
inpt = input('enter the text')
text = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inpt]), padding='pre', maxlen=200)
dictEmotion = {0: 'joy', 1: 'fear', 2: 'anger', 3: 'sadness', 4: 'disgust', 5: 'shame' , 6: 'guilt'}

print(model.predict(text), dictEmotion[np.argmax(model.predict(text))])

enter the texti am dead
[[4.4026485e-05 9.3365020e-01 7.6470867e-04 6.5280497e-02 1.6745357e-04
  7.7893041e-05 1.5167064e-05]] fear


In [78]:
tfjs.converters.save_keras_model(model, '/content/model')